In [1]:
from IPython.display import display, clear_output
from urllib.request import urlopen
import pandas as pd
import datetime
import requests
import sched
import time
import json
import urllib.request as req
import bs4 


s = sched.scheduler(time.time, time.sleep)

In [2]:
def tableColor(val):
    if val > 0:
        color = 'red'
    elif val < 0:
        color = 'green'
    else:
        color = 'white'
    return 'color: %s' % color

In [8]:
def stock_crawler(targets):
    
    clear_output(wait=True)
    
    # 組成stock_list
    stock_list = '|'.join('otc_{}.tw'.format(target) for target in targets)
    
    #　query data
    query_url = " https://mis.twse.com.tw/stock/api/getStockInfo.jsp?ex_ch="+stock_list+"&json=1&delay=0&_=1600313327541"
   
    reques=req.Request(query_url,headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36"
})
    data = json.loads(urlopen(query_url).read())
   

    # 過濾出有用到的欄位
    columns = ['c','n','z','tv','v','o','h','l','y']
    df = pd.DataFrame(data['msgArray'], columns=columns)
    df.columns = ['股票代號','公司簡稱','當盤成交價','當盤成交量','累積成交量','開盤價','最高價','最低價','昨收價']
    df.insert(9, "漲跌百分比", 0.0) 
    
    # 新增漲跌百分比
  
    
    for x in range(len(df.index)):
        if df['當盤成交價'].iloc[x] != '-':
            df.iloc[x, [2,3,4,5,6,7,8]] = df.iloc[x, [2,3,4,5,6,7,8]].astype(float)
            df.iloc[x,9] = (df['當盤成交價'].iloc[x] - df['昨收價'].iloc[x])/df['昨收價'].iloc[x] * 100
    # 紀錄更新時間
    time = datetime.datetime.now()  
    
    # show table
    df = df.style.applymap(tableColor, subset=['漲跌百分比'])
    print("更新時間:" + str(time.hour)+":"+str(time.minute))
    display(df)
    
    start_time = datetime.datetime.strptime(str(time.date())+'9:30', '%Y-%m-%d%H:%M')
    end_time =  datetime.datetime.strptime(str(time.date())+'13:30', '%Y-%m-%d%H:%M')
    # 判斷爬蟲終止條件
    if time >= start_time and time <= end_time:
        s.enter(5, 0, stock_crawler, argument=(targets,))
        s.run()

In [ ]:
stock_crawler("6203")

更新時間:11:30


,股票代號,公司簡稱,當盤成交價,當盤成交量,累積成交量,開盤價,最高價,最低價,昨收價,漲跌百分比


In [ ]:
# 欲爬取的股票代碼
stock_list = ['1101','1102','1103']

# 每秒定時器
s.enter(2, 0, stock_crawler, argument=(stock_list,))
s.run()